Limpieza para los datos 2.

In [378]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

Funciones

In [379]:
def separar_df(df):
    filtro = df['torneo'].str.contains('Men|ATP', regex=True, case = False)
    masc = df[filtro]
    return masc

def asignar_año(fecha_str):
    # Limpiar y convertir a día y mes (sin año)
    dia_mes = fecha_str.strip('/')  # Elimina '/' final
    dia, mes = map(int, dia_mes.split('/'))
    
    # Asignar 2024 si es antes de 11 de junio, 2025 en caso contrario
    if (mes < 6) or (mes == 6 and dia <= 11):
        return f"{fecha_str}2025"
    else:
        return f"{fecha_str}2024"

def df_jugador(jugador, masc, sup):

    filtro_jugador = (masc['jugadorA'] == jugador) | (masc['jugadorB'] == jugador)
    df_jugador = masc[filtro_jugador].copy()

    df_jugador_A = df_jugador[df_jugador['jugadorA'] == jugador].copy()
    df_jugador_B = df_jugador[df_jugador['jugadorB'] == jugador].copy()

    columnas_A = [col for col in df_jugador.columns if col.endswith('A')]
    columnas_B = [col for col in df_jugador.columns if col.endswith('B')]
    mapeo_columnas = {**{col_a: col_b for col_a, col_b in zip(columnas_A, columnas_B)},
                    **{col_b: col_a for col_a, col_b in zip(columnas_A, columnas_B)}}

    df_jugador_B_invertido = df_jugador_B.rename(columns=mapeo_columnas)

    df_final = pd.concat([df_jugador_A, df_jugador_B_invertido], ignore_index=True)

    df = df_final[df_final['superficie'] == sup]

    df = df[(df['setsA'] != 3) & (df['setsB'] != 3)]

    df = df[df['retirado'] == 'no']

    df = df[df['acesA'].notna() & df['acesB'].notna()]

    if 'fecha' in df.columns:
        df = df.sort_values('fecha', ascending=False)

    return df

Lectura

In [380]:
masc1 = pd.read_csv(".\\data\\data2\\acapulco_adelaide_masc_2025-06-12.csv")
masc2 = pd.read_csv(".\\data\\data2\\adelaide2-asiangames_masc_2025-06-12.csv")
masc3 = pd.read_csv(".\\data\\data2\\astana-bolzano_masc_2025-06-12.csv")
masc4 = pd.read_csv(".\\data\\data2\\bordeaux-frenchopen_masc_2025-06-12.csv")
masc5 = pd.read_csv(".\\data\\data2\\geneva-mediterraneangames_masc_2025-06-12.csv")
masc6 = pd.read_csv(".\\data\\data2\\memphis-metz_masc_2025-06-13.csv")
masc7 = pd.read_csv(".\\data\\data2\\miami-milan_masc_2025-06-13.csv")
masc8 = pd.read_csv(".\\data\\data2\\montecarlo-palermo_masc_2025-06-13.csv")
masc9 = pd.read_csv(".\\data\\data2\\paris-zhuhai_masc_2025-06-13.csv")
masc10 = pd.read_csv(".\\data\\data2\\aachen-cordoba_challenger_2025-06-15.csv", low_memory = False)
masc11 = pd.read_csv(".\\data\\data2\\cordoba2-nanjing_challenger_2025-06-16.csv", low_memory = False)
masc12 = pd.read_csv(".\\data\\data2\\napa-zug_challenger_2025-06-17.csv", low_memory = False)

dfs_masc = [masc1, masc2, masc3, masc4, masc5, masc6, masc7, masc8, masc9, masc10, masc11, masc12]
masc = pd.concat(dfs_masc, ignore_index=True)

masc = masc.drop_duplicates()

print(len(masc))

30484


In [381]:
masc = masc[(masc['setsA'] != 'Isroilov S.')]
masc = masc[(masc['setsA'] != 'Hanzlik C.')]
masc = masc[(masc['setsA'] != '6')]
masc = masc[(masc['setsB'] != '5')]
masc = masc[(masc['setsB'] != '7')]

jugadores_A = masc['jugadorA'].unique().tolist()
jugadores_B = masc['jugadorB'].unique().tolist()
todos_jugadores = list(set(jugadores_A + jugadores_B))
masc = masc[~masc['setsA'].isin(todos_jugadores)]

masc['setsA'] = masc['setsA'].astype(int)
masc['setsB'] = masc['setsB'].astype(int)

filtroGrandSlam = (masc['torneo'] == 'australian-open') | (masc['torneo'] == 'us-open') | (masc['torneo'] == 'wimbledon') | (masc['torneo'] == 'french-open')
filtro3sets = (masc['setsA'] == 3) | (masc['setsB'] == 3)
filtro2sets = (masc['setsA'] == 2) | (masc['setsB'] == 2)
filtroGrandSlamGanaA = masc['setsA'] == 3
filtroGrandSlamGanaB = masc['setsB'] == 3
filtroNoGrandSlamGanaA = masc['setsA'] == 2
filtroNoGrandSlamGanaB = masc['setsB'] == 2

# Condición 1: es Grand Slam y no se llegó a 3 sets (partido incompleto/retirado)
cond1 = filtroGrandSlam & (~filtro3sets)
masc.loc[cond1, 'retirado'] = 'si'

# Condición 2: no es Grand Slam y no se llegó a 2 sets (partido incompleto/retirado)
cond2 = (~filtroGrandSlam) & (~filtro2sets)
masc.loc[cond2, 'retirado'] = 'si'

#Creo la variable superficie
torneos_superficies = [
    ('acapulco', 'Hard'), ('adelaide', 'Hard'), ('adelaide-2', 'Hard'), ('almaty', 'Clay'), ('almaty-2', 'Clay'), ('amersfoort', 'Clay'), ('amsterdam', 'Clay'), ('antalya', 'Grass'), 
    ('antalya-2', 'Grass'), ('antalya-3', 'Grass'), ('antalya-4', 'Grass'), ('antwerp', 'Hard'), ('antwerp-2', 'Hard'), ('asian-games', 'Hard'), ('astana', 'Hard'), ('astana-2', 'Hard'), 
    ('athens', 'Clay'), ('atlanta', 'Hard'), ('atp-cup', 'Hard'), ('auckland', 'Hard'), ('australian-open', 'Hard'), ('banja-luka', 'Clay'), ('bangkok', 'Hard'), ('bangkok-2', 'Hard'), 
    ('bangkok-3', 'Hard'), ('bangkok-4', 'Hard'), ('barcelona', 'Clay'), ('barletta', 'Clay'), ('basel', 'Hard'), ('bastad', 'Clay'), ('beijing', 'Hard'), ('belgrade', 'Clay'), 
    ('belgrade-2', 'Clay'), ('bergamo', 'Hard'), ('berlin', 'Grass'), ('bermuda', 'Hard'), ('biella', 'Hard'), ('biella-2', 'Hard'), ('biella-3', 'Hard'), ('biella-4', 'Hard'), 
    ('biella-5', 'Hard'), ('biella-6', 'Hard'), ('biella-7', 'Hard'), ('biel', 'Hard'), ('birmingham', 'Grass'), ('blois', 'Clay'), ('bogota', 'Clay'), ('bogota-2', 'Clay'), 
    ('bogota-3', 'Clay'), ('bolzano', 'Hard'), ('bordeaux', 'Clay'), ('boston', 'Hard'), ('brasilia', 'Clay'), ('brighton', 'Hard'), ('brisbane', 'Hard'), ('brisbane-2', 'Hard'), 
    ('bucharest', 'Clay'), ('budapest', 'Clay'), ('buenos-aires', 'Clay'), ('buenos-aires-2', 'Clay'), ('buenos-aires-3', 'Clay'), ('buzios', 'Clay'), ('cagliari', 'Clay'), ('calgary', 'Hard'), 
    ('campinas', 'Clay'), ('canberra', 'Hard'), ('canberra-2', 'Hard'), ('cap-cana', 'Hard'), ('casablanca', 'Clay'), ('casablanca-2', 'Clay'), ('cassis', 'Hard'), ('cary', 'Hard'), 
    ('cary-2', 'Hard'), ('champaign', 'Hard'), ('chengdu', 'Hard'), ('chennai', 'Hard'), ('chicago', 'Hard'), ('chitre', 'Hard'), ('chongqing', 'Hard'), ('cincinnati', 'Hard'), 
    ('cleveland', 'Hard'), ('cologne', 'Hard'), ('cologne-2', 'Hard'), ('como', 'Clay'), ('concepcion', 'Clay'), ('concepcion-2', 'Clay'), ('constanta', 'Clay'), ('copenhagen', 'Hard'), 
    ('cordoba', 'Clay'), ('cordoba-2', 'Clay'), ('corrientes', 'Clay'), ('costa-do-sauipe', 'Clay'), ('courmayeur', 'Hard'), ('curitiba', 'Clay'), ('dallas', 'Hard'), ('delray-beach', 'Hard'), 
    ('doha', 'Hard'), ('drummondville', 'Hard'), ('dubai', 'Hard'), ('dusseldorf', 'Clay'), ('eckental', 'Carpet'), ('eastbourne', 'Grass'), ('essen', 'Hard'), ('estoril', 'Clay'), 
    ('fergana', 'Hard'), ('florence', 'Hard'), ('floridablanca', 'Clay'), ('forli', 'Clay'), ('forli-2', 'Clay'), ('forli-3', 'Clay'), ('forli-4', 'Clay'), ('forli-5', 'Clay'), 
    ('forli-6', 'Clay'), ('francavilla', 'Clay'), ('french-open', 'Clay'), ('furth', 'Clay'), ('gijon', 'Hard'), ('gimcheon', 'Hard'), ('glasgow', 'Hard'), ('granby', 'Hard'), 
    ('gran-canaria', 'Hard'), ('gran-canaria-2', 'Hard'), ('grodzisk-mazowiecki', 'Hard'), ('gstaad', 'Clay'), ('guadalajara', 'Hard'), ('guangzhou', 'Hard'), ('guangzhou-2', 'Hard'), 
    ('guayaquil', 'Clay'), ('guimaraes', 'Hard'), ('gwangju', 'Hard'), ('halle', 'Grass'), ('hamilton', 'Hard'), ('hamburg', 'Clay'), ('hangzhou', 'Hard'), ('happy-valley', 'Hard'), 
    ('helsinki', 'Hard'), ('heilbronn', 'Hard'), ('heilbronn-2', 'Hard'), ('hertogenbosch', 'Grass'), ('ho-chi-minh-city', 'Hard'), ('hong-kong', 'Hard'), ('houston', 'Clay'), ('ibague', 'Clay'), 
    ('indian-wells', 'Hard'), ('indian-wells-2', 'Hard'), ('irving', 'Hard'), ('islamabad', 'Hard'), ('istanbul', 'Clay'), ('istanbul-2', 'Clay'), ('itajai', 'Clay'), ('jakarta', 'Hard'), 
    ('jerusalem', 'Hard'), ('jinan', 'Hard'), ('karlsruhe', 'Clay'), ('kazan', 'Hard'), ('kazan-2', 'Hard'), ('kenitra', 'Clay'), ('kigali', 'Hard'), ('kigali-2', 'Hard'), ('kitzbuhel', 'Clay'), ('koblenz', 'Hard'), 
    ('kolding', 'Hard'), ('kolkata', 'Hard'), ('kosice', 'Clay'), ('kyoto', 'Hard'), ('launceston', 'Hard'), ('lille', 'Hard'), ('lima', 'Clay'), ('lima-2', 'Clay'), ('lisbon', 'Hard'), ('little-rock', 'Hard'), 
    ('ljubljana', 'Hard'), ('london', 'Grass'), ('los-cabos', 'Hard'), ('luedenscheid', 'Clay'), ('lugano', 'Clay'), ('lyon', 'Hard'), ('lyon-2', 'Hard'), ('madrid', 'Clay'), ('madrid-2', 'Clay'), 
    ('maia', 'Clay'), ('maia-2', 'Clay'), ('mallorca', 'Grass'), ('manama', 'Hard'), ('manchester', 'Grass'), ('manerbio', 'Clay'), ('marrakech', 'Clay'), ('marseille', 'Hard'), 
    ('matsuyama', 'Hard'), ('mauthausen', 'Clay'), ('medellin', 'Clay'), ('melbourne', 'Hard'), ('menorca', 'Hard'), ('mersin', 'Clay'), ('merida', 'Hard'), ('metz', 'Hard'), 
    ('mexico-city', 'Clay'), ('milan', 'Hard'), ('minsk', 'Hard'), ('modena', 'Clay'), ('monte-carlo', 'Clay'), ('monterrey', 'Hard'), ('montpellier', 'Hard'), ('montreal', 'Hard'), 
    ('moscow', 'Clay'), ('mouilleron-le-captif', 'Hard'), ('munich', 'Clay'), ('murcia', 'Clay'), ('napa', 'Hard'), ('napoli', 'Clay'), ('napoli-2', 'Clay'), ('new-york', 'Hard'), 
    ('newport', 'Grass'), ('newport-beach', 'Hard'), ('nice', 'Clay'), ('ningbo', 'Hard'), ('nonthaburi', 'Hard'), ('nonthaburi-2', 'Hard'), ('nonthaburi-3', 'Hard'), ('nonthaburi-4', 'Hard'), 
    ('noumea', 'Hard'), ('nur-sultan', 'Hard'), ('nur-sultan-2', 'Hard'), ('nur-sultan-3', 'Hard'), ('nur-sultan-4', 'Hard'), ('oeiras', 'Clay'), ('oeiras-2', 'Clay'), ('oeiras-3', 'Clay'), 
    ('oeiras-4', 'Clay'), ('oeiras-5', 'Clay'), ('oeiras-6', 'Clay'), ('olbia', 'Hard'), ('olympic-games', 'Hard'), ('oporto', 'Hard'), ('orleans', 'Hard'), ('orlando', 'Hard'), 
    ('orlando-2', 'Hard'), ('osaka', 'Hard'), ('ostrava', 'Hard'), ('padova', 'Clay'), ('palermo', 'Clay'), ('pau', 'Hard'), ('pereira', 'Clay'), ('perugia', 'Clay'), ('phoenix', 'Hard'), ('piracicaba', 'Clay'), 
    ('playford', 'Hard'), ('playford-2', 'Hard'), ('poertschach', 'Clay'), ('poznan', 'Clay'), ('prague', 'Clay'), ('prague-3', 'Clay'), ('prostejov', 'Clay'), ('puebla', 'Hard'), ('pune', 'Hard'), ('punta-del-este', 'Clay'), 
    ('quito', 'Clay'), ('rabat', 'Clay'), ('raanana', 'Hard'), ('recanati', 'Clay'), ('reggio-emilia', 'Clay'), ('rennes', 'Hard'), ('rhodes', 'Hard'), ('rio-de-janeiro', 'Clay'), 
    ('rio-de-janeiro-2', 'Clay'), ('rimini', 'Clay'), ('rio-quente', 'Hard'), ('roanne', 'Hard'), ('roseto-degli-abruzzi', 'Clay'), ('roseto-degli-abruzzi-2', 'Clay'), ('rotterdam', 'Hard'), 
    ('rovereto', 'Hard'), ('saint-brieuc', 'Hard'), ('salinas', 'Hard'), ('salinas-2', 'Hard'), ('salzburg', 'Clay'), ('san-diego', 'Hard'), ('san-juan', 'Hard'), ('san-luis-potosi', 'Hard'), 
    ('san-marino', 'Clay'), ('san-miguel-de-tucuman', 'Clay'), ('san-remo', 'Clay'), ('san-sebastian', 'Clay'), ('santos', 'Clay'), ('santos-open', 'Clay'), ('sao-paulo', 'Clay'), 
    ('sao-paulo-2', 'Clay'), ('sao-paulo-3', 'Clay'), ('sao-paulo-4', 'Clay'), ('sao-paulo-5', 'Clay'), ('sarajevo', 'Hard'), ('sarasota', 'Clay'), ('santiago', 'Clay'), ('santiago-2', 'Clay'), 
    ('santiago-3', 'Clay'), ('santo-domingo', 'Hard'), ('santos', 'Clay'), ('sardinia', 'Clay'), ('savannah', 'Hard'), ('schenectady', 'Hard'), ('scheveningen', 'Clay'), ('scottsdale', 'Hard'), 
    ('segovia', 'Hard'), ('seoul', 'Hard'), ('shenzhen', 'Hard'), ('shenzhen-2', 'Hard'), ('shenzhen-3', 'Hard'), ('shymkent', 'Hard'), ('shymkent-2', 'Hard'), ('singapore', 'Hard'), 
    ('skopje', 'Clay'), ('sofia', 'Hard'), ('sopot', 'Clay'), ('split', 'Clay'), ('split-2', 'Clay'), ('st-brieuc', 'Hard'), ('stockholm', 'Hard'), ('stockton', 'Hard'), ('surbiton', 'Grass'), 
    ('sydney', 'Hard'), ('sydney-2', 'Hard'), ('szczecin', 'Clay'), ('szekesfehervar', 'Clay'), ('taipei', 'Hard'), ('taipei-2', 'Hard'), ('tallahassee', 'Hard'), ('tampere', 'Hard'), 
    ('tanger', 'Clay'), ('tarragona', 'Clay'), ('tashkent', 'Hard'), ('tel-aviv', 'Hard'), ('tenerife', 'Hard'), ('tenerife-2', 'Hard'), ('tenerife-3', 'Hard'), ('tianjin', 'Hard'), 
    ('tiburon', 'Hard'), ('timisoara', 'Clay'), ('tokyo', 'Hard'), ('tokyo-2', 'Hard'), ('toulouse', 'Hard'),('sassuolo', 'Clay'), ('royan', 'Clay'), ('poznan', 'Clay'), ('nottingham', 'Grass'), 
    ('santa-cruz', 'Hard'), ('ilkley', 'Grass'), ('bratislava', 'Clay'), ('santa-fe', 'Clay'), ('perugia', 'Clay'), ('stuttgart', 'Grass'), ('hertogenbosch', 'Grass'), ('tyler', 'Hard'), 
    ('prostejov', 'Clay'), ('chisinau', 'Clay'), ('vicenza', 'Clay'), ('tbilisi', 'Hard'), ('geneva', 'Clay'), ('turin-2', 'Hard'), ('rome', 'Clay'), ('tunis', 'Clay'), 
    ('zagreb', 'Hard'), ('wuxi', 'Hard'), ('francavilla', 'Clay'), ('ostrava', 'Hard'), ('porto-alegre', 'Clay'), ('aix-en-provence', 'Clay'), ('gwangju', 'Hard'), ('abidjan-2', 'Hard'), 
    ('savannah', 'Hard'), ('abidjan', 'Hard'), ('busan', 'Hard'), ('tallahassee', 'Hard'), ('monza', 'Clay'), ('mexico-city', 'Hard'), ('houston', 'Clay'), ('morelos', 'Hard'), 
    ('menorca', 'Hard'), ('girona', 'Clay'), ('naples', 'Clay'), ('merida', 'Hard'), ('zadar', 'Clay'), ('asuncion', 'Clay'), ('murcia', 'Clay'), ('phoenix', 'Hard'), 
    ('hersonissos-2', 'Hard'), ('cherbourg', 'Hard'), ('hersonissos', 'Hard'), ('thionville', 'Hard'), ('lugano', 'Clay'), ('san-diego', 'Hard'), ('brazzaville', 'Hard'), ('pune', 'Hard'), 
    ('new-delhi', 'Hard'), ('tenerife-2', 'Hard'), ('manama', 'Hard'), ('rosario', 'Clay'), ('tenerife', 'Hard'), ('quimper', 'Hard'), ('oeiras-5', 'Clay'), ('oeiras-2', 'Clay'), 
    ('nonthaburi-3', 'Hard'), ('nottingham-3', 'Grass'), ('nonthaburi-2', 'Hard'), ('hong-kong', 'Hard'), ('united-cup', 'Hard'), ('noumea', 'Hard'), ('manzanillo', 'Hard'), ('yokkaichi', 'Hard'), 
    ('temuco', 'Clay'), ('puerto-vallarta', 'Hard'), ('yokohama', 'Hard'), ('montevideo', 'Clay'), ('kobe', 'Hard'), ('matsuyama', 'Hard'), ('knoxville', 'Hard'), ('guayaquil', 'Clay'), 
    ('charlottesville', 'Hard'), ('bratislava-2', 'Clay'), ('playford-2', 'Hard'), ('sioux-falls', 'Hard'), ('brest', 'Hard'), ('olbia', 'Hard'), ('shenzhen-2', 'Hard'), ('villa-maria', 'Clay'), 
    ('hangzhou', 'Hard'), ('mouilleron-le-captif', 'Hard'), ('alicante', 'Clay'), ('braga', 'Clay'), ('charleston', 'Clay'), ('antofagasta', 'Clay'), ('nonthaburi-4', 'Hard'), ('cali', 'Clay'), 
    ('laver-cup', 'Hard'), ('sibiu', 'Clay'), ('columbus', 'Hard'), ('las-vegas', 'Hard'), ('dobrich-2', 'Clay'), ('genova', 'Clay'), ('tulln', 'Clay'), ('seville', 'Clay'), 
    ('porto-2', 'Hard'), ('winston-salem', 'Hard'), ('dobrich', 'Clay'), ('santo-domingo', 'Hard'), ('grodzisk-mazowiecki', 'Hard'), ('todi', 'Clay'), ('lincoln', 'Hard'), ('bonn', 'Hard'), 
    ('cordenons', 'Clay'), ('liberec', 'Clay'), ('lexington', 'Hard'), ('luedenscheid', 'Clay'), ('verona', 'Clay'), ('segovia', 'Hard'), ('zug', 'Hard'), ('granby', 'Hard'), 
    ('pozoblanco', 'Hard'), ('salzburg', 'Clay'), ('iasi', 'Clay'), ('trieste', 'Clay'),  ('winnipeg', 'Hard'), ('braunschweig', 'Clay'), ('brasov', 'Clay'), ('troyes', 'Clay'), 
    ('bloomfield-hills', 'Hard'), ('ibague', 'Clay'), ('santa-cruz-2', 'Hard'), ('surbiton', 'Grass'), ('kachreti', 'Hard'), ('augsburg', 'Hard'), ('oeiras-4', 'Clay'), ('shenzhen-3', 'Hard'), 
    ('split', 'Clay'), ('florianopolis', 'Clay'), ('sao-leopoldo', 'Clay'), ('szekesfehervar', 'Clay'), ('los-cabos', 'Hard'), ('burnie-2', 'Hard'), ('burnie', 'Hard'), ('ottignies-louvain-la-neuve', 'Hard'), 
    ('maspalomas', 'Hard'), ('danderyd', 'Hard'), ('ortisei', 'Hard'), ('malaga', 'Hard'), ('zhuhai', 'Hard'), ('prague-3', 'Clay'), ('stanford', 'Hard'), ('meerbusch', 'Clay'), 
    ('salinas', 'Hard'), ('san-benedetto', 'Clay'), ('palmas-del-mar', 'Hard'), ('troisdorf', 'Hard'), ('roseto-degli-abruzzi', 'Clay'), ('sanremo', 'Clay'), ('las-franquesas-del-valles', 'Clay'), 
    ('vina-del-mar', 'Clay'), ('waco', 'Hard'), ('rome-us', 'Hard'), ('vilnius', 'Hard'), ('tigre-2', 'Clay'), ('tigre', 'Clay'), ('andria', 'Hard'), ('ambato', 'Clay'), 
    ('gijon', 'Hard'), ('parma-2', 'Clay'), ('tel-aviv', 'Hard'), ('columbus-2', 'Hard'), ('vancouver', 'Hard'), ('indianapolis', 'Hard'), ('orlando', 'Hard'), ('forli-6', 'Clay'), 
    ('shymkent-2', 'Hard'), ('shymkent', 'Hard'), ('aguascalientes', 'Clay'), ('marbella', 'Clay'), ('concepcion-2', 'Clay'), ('gran-canaria', 'Hard'), ('forli-5', 'Clay'), ('forli-4', 'Clay'), 
    ('forli-3', 'Clay'), ('blumenau', 'Clay'), ('forli-2', 'Clay'), ('traralgon', 'Hard'), ('bendigo', 'Hard'), ('bari', 'Clay'), ('moscow', 'Clay'), ('losinj', 'Clay'), 
    ('quito', 'Clay'), ('kyiv', 'Hard'), ('warsaw', 'Clay'), ('nottingham-2', 'Grass'), ('singapore', 'Hard'), ('potchefstroom-2', 'Hard'), ('potchefstroom', 'Hard'), ('quimper-2', 'Hard'), 
    ('sardinia', 'Clay'), ('new-york', 'Hard'), ('launceston', 'Hard'), ('ann-arbor', 'Hard'), ('liuzhou', 'Hard'), ('ningbo', 'Hard'), ('columbus-3', 'Hard'), ('kaohsiung', 'Hard'), 
    ('baotou', 'Hard'), ('portoroz', 'Hard'), ('aptos', 'Hard'), ('manerbio', 'Clay'), ('binghamton', 'Hard'), ('gatineau', 'Hard'), ('winnetka', 'Hard'), ('fergana', 'Hard'), 
    ('samarkand', 'Hard'), ('an-ning', 'Clay'), ('sophia-antipolis', 'Hard'), ('da-nang', 'Hard'), ('pullach', 'Clay'), ('marburg', 'Clay'), ('poprad', 'Clay'), ('caltanissetta', 'Clay'), 
    ('mestre', 'Clay'), ('loughborough', 'Hard'), ('guadalajara', 'Hard'), ('panama-city', 'Hard'), ('qujing', 'Hard'), ('kyoto', 'Hard'), ('san-francisco', 'Hard'), ('hua-hin', 'Hard'), 
    ('toyota', 'Hard'), ('suzhou', 'Hard'), ('alphen', 'Clay'), ('cortina', 'Clay'), ('qingdao', 'Hard'), ('quanzhou', 'Hard'), ('wroclaw', 'Hard'), ('tempe', 'Hard'), 
    ('happy-valley', 'Hard'), ('mons', 'Hard'), ('mohammedia', 'Clay'), ('kenitra', 'Clay'), ('meknes', 'Clay'), ('barranquilla', 'Clay'), ('fano', 'Clay'), ('trnava', 'Clay'), 
    ('manchester', 'Grass'), ('nice', 'Clay'), ('nanjing', 'Hard'), ('raanana', 'Hard'), ('jonkoping', 'Hard'), ('manila', 'Hard'), ('mendoza', 'Clay'), ('sacramento', 'Hard'), 
    ('kuala-lumpur', 'Hard'), ('agri', 'Clay'), ('eskisehir', 'Hard'), ('vercelli', 'Clay'), ('batman', 'Clay'), ('kolkata', 'Hard'), ('traralgon-2', 'Hard'), ('reunion-island', 'Hard'), 
    ('indore', 'Hard'), ('napa', 'Hard'), ('oberstaufen', 'Clay'), ('manta', 'Hard'), ('kosice', 'Clay'), ('arad', 'Clay'), ('rimouski', 'Hard'), ('chitre', 'Hard'), 
    ('tyumen', 'Hard'), ('yeongwol', 'Hard'), ('petange', 'Hard'), ('timisoara', 'Clay'), ('kun-ming', 'Hard'), ('rio-quente', 'Hard'), ('sarajevo', 'Hard'), ('villa-allende', 'Clay'), 
    ('lermontov', 'Clay'), ('wuhan', 'Hard'), ('bercuit', 'Clay'), ('penza', 'Hard'), ('rabat', 'Clay'), ('wolfsburg', 'Hard'), ('caloundra', 'Hard'), ('palermo', 'Clay'), ('belo-horizonte', 'Clay'),
    ('tulsa', 'Hard'), ('trani', 'Clay'), ('dortmund', 'Clay'), ('rijeka', 'Hard'), ('alessandria', 'Clay'), ('courmayeur', 'Hard'), ('cancun', 'Hard'), 
    ('tarragona', 'Clay'), ('salvador', 'Hard'), ('rimini', 'Clay'), ('bytom', 'Clay'), ('rome-3', 'Clay'), ('ojai', 'Hard'), ('san-remo', 'Clay'), ('cairo', 'Clay'), 
    ('sunrise', 'Hard'), ('jersey-2', 'Hard'), ('freudenstadt', 'Clay'), ('constanta', 'Clay'), ('korat', 'Hard'), ('iquique', 'Hard'), ('poertschach', 'Clay'), ('long-island', 'Hard'), 
    ('stuttgart-1', 'Hard'), ('verizon-tennis-challenge', 'Hard'), ('grand-slam-cup', 'Hard'), ('philadelphia', 'Hard'), ('merano', 'Clay'), ('schenectady', 'Hard'), ('pinehurst', 'Clay'), 
    ('osaka', 'Hard'), ('oahu', 'Hard'), ('genoa', 'Clay'), ('wembley', 'Hard'), ('itaparica', 'Hard'), ('toronto-2', 'Hard'),
    ('miami', 'Hard'), ('morelia', 'Clay'), ('bengaluru', 'Hard'), ('montemar', 'Clay'), ('paris', 'Hard'), ('vienna', 'Hard'), ('valencia', 'Clay'), ('shanghai', 'Hard'), 
    ('fairfield', 'Hard'), ('bad-waltersdorf', 'Clay'), ('us-open', 'Hard'), ('zhangjiagang', 'Hard'), ('washington', 'Hard'), ('porto', 'Clay'), ('umag', 'Clay'), 
    ('wimbledon', 'Grass'), ('ismaning', 'Carpet'), ('santa-fe-2', 'Clay'), ('toronto', 'Hard'), ('hopman-cup', 'Hard'), ('parma', 'Clay'), ('coquimbo', 'Clay'), ('coquimbo-2', 'Clay'), 
    ('salvador-de-bahia', 'Clay'), ('bengaluru-2', 'Hard'), ('rome-2', 'Clay'), ('new-haven', 'Hard'), ('ludwigshafen', 'Clay'), ('nanchang', 'Hard'), ('karshi', 'Hard'), 
    ('le-gosier', 'Hard'), ('brescia', 'Clay'), ('izmir', 'Hard'), ('memphis', 'Hard'), ('turin', 'Hard'), ('atp-challenger-tour-finals', 'Hard'), ('cali-2', 'Clay'), 
    ('sao-jose-do-rio-preto', 'Clay'), ('orbetello', 'Clay'), ('johannesburg', 'Hard'), ('san-jose', 'Hard'), ('belem', 'Clay'), ('los-angeles', 'Hard'), ('bath', 'Hard'), 
    ('pingguo', 'Hard'), ('honolulu', 'Hard'), ('recife-2', 'Clay'), ('campos-do-jordao', 'Hard'), ('wuhai', 'Hard'), ('cremona', 'Clay'), ('recife', 'Clay'), ('sainte-clotilde', 'Hard'), 
    ('aachen', 'Clay'), ('calabasas', 'Hard'), ('saransk', 'Clay'), ('carson', 'Hard'), ('ramat-hasharon', 'Hard'), ('baton-rouge', 'Hard'), ('jersey', 'Hard'), 
    ('khanty-mansiysk', 'Hard'), ('chuncheon', 'Hard'), ('vigo', 'Clay'), ('mediterranean-games', 'Clay'), ('yuba-city', 'Hard'), ('karlsruher', 'Clay'), 
    ('carson-2', 'Hard'), ('guinot-mary-cohr', 'Hard'), ('besancon', 'Hard'), ('dnepropetrovsk', 'Hard'), ('mumbai', 'Hard'), ('shanghai-2', 'Hard'), 
    ('new-haven-2', 'Hard'), ('bologna', 'Clay'), ('kuala-lumpur-2', 'Hard'), ('zaragoza', 'Clay'), ('wellington', 'Hard'), ('guaruja', 'Clay'), ('maceio', 'Clay')

]

superficie_dict = dict(torneos_superficies)
masc['superficie'] = masc['torneo'].map(superficie_dict)

masc.loc[masc['torneo'] == 'memphis', 'ano'] = 2017
masc.loc[masc['torneo'] == 'merano', 'ano'] = 1997
masc.loc[masc['torneo'] == 'metz', 'ano'] = 2024
masc[['dia', 'hora']] = masc['fecha'].str.split(' ', expand=True) #pongo la fecha como de normal (13/01/2025)
masc['dia'] = masc['dia'].str.replace('.', '/', regex=False)
masc['ano'] = masc['ano'].apply(lambda x: str(int(x)) if pd.notna(x) else x)
masc['dia'] = masc['dia'].astype(str).str.strip()
masc['ano'] = masc['ano'].astype(str).str.strip()
masc['fecha'] = masc['dia'] + masc['ano']
masc = masc.drop(columns = ['dia', 'cuotaA', 'cuotaB'])
masc['fecha'] = pd.to_datetime(masc['fecha'], format='%d/%m/%Y')

filtro = (masc['fecha'].dt.year == 2025) & (masc['fecha'].dt.month == 12)
masc.loc[filtro, 'fecha'] = masc.loc[filtro, 'fecha'] - pd.DateOffset(years=1)

masc['sets'] = masc['setsA'] + masc['setsB']

#Me quedo con las cuotas buenas
masc = masc.rename(columns={
    'cuotaA_2': 'cuotaA',
    'cuotaB_2': 'cuotaB'
})

#Le quito el símbolo del porcentaje a las columnas que lo tienen
cols_porcentaje = [col for col in masc.columns if 'porcentaje' in col.lower()]
for col in cols_porcentaje:
    masc[col] = masc[col].astype(str).str.replace('%', '', regex=False)
    masc[col] = pd.to_numeric(masc[col], errors='coerce')

#Le quito los paréntesis a las columnas que lo tienen
cols_con_par = [col for col in masc.columns if masc[col].astype(str).str.contains(r'\(\d+/\d+\)').any()]
for col in cols_con_par:
    masc[col] = masc[col].astype(str).str.replace('(', '', regex=False).str.replace(')', '', regex=False)
    
#Creo la columna ganador

masc['ganador'] = -1

ganadorA1 = filtroGrandSlam & filtroGrandSlamGanaA
masc.loc[ganadorA1, 'ganador'] = 0

ganadorB1 = filtroGrandSlam & filtroGrandSlamGanaB
masc.loc[ganadorB1, 'ganador'] = 1

ganadorA2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaA
masc.loc[ganadorA2, 'ganador'] = 0

ganadorB2 = (~filtroGrandSlam) & filtroNoGrandSlamGanaB
masc.loc[ganadorB2, 'ganador'] = 1

masc = masc[masc['acesA'] != '31.05.2025 00:16']
masc = masc.sort_values('fecha', ascending=False).reset_index(drop=True)

masc.head()


,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,dobleFaltasA,...,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,ano,PuntosGanadosRedA,PuntosGanadosRedB,superficie,hora,sets,ganador
0,sassuolo,Roca Batalla O.,Fonio G.,0,2,2025-06-16,no,5,4,NaN,...,2/10,8/20,12/20,2025,nan,nan,Clay,11:30,2,1
1,royan,Boitan G. A.,Couacaud E.,1,2,2025-06-16,no,6,15,NaN,...,2/14,13/28,15/28,2025,nan,nan,Clay,14:35,3,1
2,sassuolo,Bocchi L.,Nijboer R.,0,2,2025-06-16,no,3,2,NaN,...,4/9,6/18,12/18,2025,nan,nan,Clay,12:35,2,1
3,sassuolo,Ymer E.,Brancaccio R.,2,1,2025-06-16,no,2,2,NaN,...,6/13,12/26,14/26,2025,nan,nan,Clay,11:55,3,0
4,sassuolo,Martin A.,Cecchinato M.,0,2,2025-06-16,no,2,1,NaN,...,4/9,7/19,12/19,2025,nan,nan,Clay,10:10,2,1


Ahora vamos a tratar los nan

In [382]:
filtro = masc.isna().sum() == 0
print(len(masc.isna().sum()[filtro])) #hay 37 columnas con 0 nan
print(masc.isna().sum()[filtro].index.tolist())

38
['torneo', 'jugadorA', 'jugadorB', 'setsA', 'setsB', 'fecha', 'retirado', 'juegosGanadosPrimerSaqueA', 'juegosGanadosPrimerSaqueB', 'JuegosGanadosSegundoSaqueA', 'JuegosGanadosSegundoSaqueB', 'BreakPointsSalvadosA', 'BreakPointsSalvadosB', 'PuntosGanadosAlRestoConPrimerSaqueA', 'PuntosGanadosAlRestoConPrimerSaqueB', 'PuntosGanadosAlRestoA', 'PuntosGanadosAlRestoB', 'PuntosGanadosAlRestoConSegundoSaqueA', 'PuntosGanadosAlRestoConSegundoSaqueB', 'PuntosBreakConvertidosA', 'PuntosBreakConvertidosB', 'PuntosGanadosAlSaqueA', 'PuntosGanadosAlSaqueB', 'PuntosGanadosA', 'PuntosGanadosB', 'JuegosGanadosAlSaqueA', 'JuegosGanadosAlSaqueB', 'JuegosGanadosAlRestoA', 'JuegosGanadosAlRestoB', 'JuegosGanadosA', 'JuegosGanadosB', 'ano', 'PuntosGanadosRedA', 'PuntosGanadosRedB', 'superficie', 'hora', 'sets', 'ganador']


In [383]:
filtro = (masc.isna().sum() > 0) & (masc.isna().sum() <= 3000)
masc.isna().sum()[filtro]
print(masc.isna().sum()[filtro].index.tolist())

['cuotaA', 'cuotaB']


In [384]:
filtro = (masc.isna().sum() > 3000) & (masc.isna().sum() <= 10200)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
#masc = masc.drop(columns = columnas_a_eliminar)

30
['acesA', 'acesB', 'porcentajePrimerSaqueA', 'porcentajePrimerSaqueB', 'porcentajeBreakPointsSalvadosA', 'porcentajeBreakPointsSalvadosB', 'porcentajePuntosGanadosAlRestoConPrimerSaqueA', 'porcentajePuntosGanadosAlRestoConPrimerSaqueB', 'porcentajePuntosGanadosAlRestoConSegundoSaqueA', 'porcentajePuntosGanadosAlRestoConSegundoSaqueB', 'porcentajePuntosGanadosA', 'porcentajePuntosGanadosB', 'porcentajeJuegosGanadosA', 'porcentajeJuegosGanadosB', 'dobleA', 'dobleB', 'porcentajeJuegosGanadosPrimerSaqueA', 'porcentajeJuegosGanadosPrimerSaqueB', 'porcentajeJuegosGanadosSegundoSaqueA', 'porcentajeJuegosGanadosSegundoSaqueB', 'porcentajePuntosGanadosAlRestoA', 'porcentajePuntosGanadosAlRestoB', 'porcentajePuntosBreakConvertidosA', 'porcentajePuntosBreakConvertidosB', 'porcentajePuntosGanadosAlSaqueA', 'porcentajePuntosGanadosAlSaqueB', 'porcentajeJuegosGanadosAlSaqueA', 'porcentajeJuegosGanadosAlSaqueB', 'porcentajeJuegosGanadosAlRestoA', 'porcentajeJuegosGanadosAlRestoB']


In [385]:
filtro = (masc.isna().sum() > 10200) & (masc.isna().sum() <= 15900)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
#masc = masc.drop(columns = columnas_a_eliminar)

4
['PuntosGanadosUltimas10BolasA', 'PuntosGanadosUltimas10BolasB', 'BolasDePartidoSalvadasA', 'BolasDePartidoSalvadasB']


In [386]:
filtro = (masc.isna().sum() > 15900) & (masc.isna().sum() <= 29000)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
#masc = masc.drop(columns = columnas_a_eliminar)

10
['velocidadMediaPrimerSaqueA', 'velocidadMediaPrimerSaqueB', 'velocidadMediaSegundoSaqueA', 'velocidadMediaSegundoSaqueB', 'winnersA', 'winnersB', 'erroresNoForzadosA', 'erroresNoForzadosB', 'porcentajePuntosGanadosRedA', 'porcentajePuntosGanadosRedB']


In [387]:
filtro = (masc.isna().sum() > 29000)
print(len(masc.isna().sum()[filtro]))
columnas_a_eliminar = masc.isna().sum()[filtro].index.tolist()
print(columnas_a_eliminar)
masc = masc.drop(columns = columnas_a_eliminar)

46
['dobleFaltasA', 'dobleFaltasB', 'porcentajePuntosGanadosConPrimerSaqueA', 'porcentajePuntosGanadosConPrimerSaqueB', 'puntosGanadosConPrimerSaqueA', 'puntosGanadosConPrimerSaqueB', 'porcentajePuntosGanadosConSegundoSaqueA', 'porcentajePuntosGanadosConSegundoSaqueB', 'puntosGanadosConSegundoSaqueA', 'puntosGanadosConSegundoSaqueB', 'breakPointsSalvadosA', 'breakPointsSalvadosB', 'porcentajeBreakPointsA', 'porcentajeBreakPointsB', 'breakPointsA', 'breakPointsB', 'puntosGanadosAlRestoConPrimerSaqueA', 'puntosGanadosAlRestoConPrimerSaqueB', 'puntosGanadosAlRestoConSegundoSaqueA', 'puntosGanadosAlRestoConSegundoSaqueB', 'puntosGanadosRedA', 'puntosGanadosRedB', 'porcentajePuntosGanadosSaqueA', 'porcentajePuntosGanadosSaqueB', 'puntosGanadosSaqueA', 'puntosGanadosSaqueB', 'porcentajePuntosGanadosRestoA', 'porcentajePuntosGanadosRestoB', 'puntosGanadosRestoA', 'puntosGanadosRestoB', 'puntosGanadosA', 'puntosGanadosB', 'puntosGanadosUltimas10BolasA', 'puntosGanadosUltimas10BolasB', 'puntosD

Vemos como se ha quedado el df

In [388]:
masc.shape

(30310, 84)

In [393]:
jugadorA = "Michelsen A."
jugadorB = 'Tsitsipas S.'
superf = 'Grass'

dfA = df_jugador(jugadorA, masc, superf)
dfB = df_jugador(jugadorB, masc, superf)
display(dfA)
display(dfB)

,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,porcentajePrimerSaqueA,...,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,ano,PuntosGanadosRedA,PuntosGanadosRedB,superficie,hora,sets,ganador
0,stuttgart,Michelsen A.,Engel J.,0,2,2025-06-12,no,10,5,61.0,...,2/10,8/20,12/20,2025,6/11,5/5,Grass,11:10,2,1
1,stuttgart,Michelsen A.,Monfils G.,2,1,2025-06-10,no,6,14,70.0,...,2/15,16/29,13/29,2025,24/28,0/3,Grass,18:35,3,0
38,newport,Michelsen A.,Giron M.,1,2,2024-07-21,no,0,13,70.0,...,3/16,14/33,19/33,2024,36/53,22/29,Grass,20:40,3,1
76,newport,Michelsen A.,Opelka R.,2,0,2024-07-20,no,1,8,66.0,...,0/7,12/14,2/14,2024,8/10,6/9,Grass,19:10,2,1
39,newport,Michelsen A.,Kovacevic A.,2,0,2024-07-18,no,5,12,68.0,...,0/10,12/20,8/20,2024,15/21,5/15,Grass,21:00,2,0
40,newport,Michelsen A.,Cressy M.,2,0,2024-07-17,no,1,6,63.0,...,0/10,12/20,8/20,2024,11/18,17/34,Grass,17:35,2,0
41,mallorca,Michelsen A.,Ofner S.,1,2,2024-06-27,no,5.0,12.0,74.0,...,2/15,14/30,16/30,2024,8/11,10/15,Grass,16:20,3,1
78,mallorca,Michelsen A.,Mannarino A.,2,0,2024-06-25,no,4.0,6.0,71.0,...,2/11,12/22,10/22,2024,13/21,14/28,Grass,14:00,2,1
79,mallorca,Michelsen A.,Borges N.,2,0,2024-06-23,no,3.0,10.0,82.0,...,0/11,12/21,9/21,2024,22/29,12/17,Grass,13:10,2,1
80,halle,Michelsen A.,Berrettini M.,0,2,2024-06-19,no,5.0,13.0,73.0,...,3/10,8/20,12/20,2024,nan,nan,Grass,12:10,2,0


,torneo,jugadorA,jugadorB,setsA,setsB,fecha,retirado,acesA,acesB,porcentajePrimerSaqueA,...,JuegosGanadosAlRestoB,JuegosGanadosA,JuegosGanadosB,ano,PuntosGanadosRedA,PuntosGanadosRedB,superficie,hora,sets,ganador
68,halle,Tsitsipas S.,Struff J-L.,0,2,2024-06-20,no,5.0,8.0,60.0,...,2/10,8/20,12/20,2024,nan,nan,Grass,18:35,2,0
69,halle,Tsitsipas S.,Squire H.,2,0,2024-06-18,no,7.0,6.0,65.0,...,0/12,12/24,12/24,2024,nan,nan,Grass,19:15,2,1


In [ ]:
print("Media de aces realizados por", jugadorA, "en los últimos", len(dfA),"partidos en", superf, ":",dfA['acesA'].astype(int).mean())
print("Media de aces recibidos por", jugadorA, "en los últimos", len(dfA),"partidos en", superf, ":",dfA['acesB'].astype(int).mean())
print("------------------------")
print("Media de aces realizados por", jugadorB, "en los últimos", len(dfB),"partidos en", superf, ":",dfB['acesA'].astype(int).mean())
print("Media de aces recibidos por", jugadorB, "en los últimos", len(dfB),"partidos en", superf, ":",dfB['acesB'].astype(int).mean())

Media de aces realizados por Michelsen A. en los últimos 13 partidos en Grass : 5.153846153846154
Media de aces recibidos por Michelsen A. en los últimos 13 partidos en Grass : 8.846153846153847
------------------------
Media de aces realizados por Tsitsipas S. en los últimos 2 partidos en Grass : 6.0
Media de aces recibidos por Tsitsipas S. en los últimos 2 partidos en Grass : 7.0
